How to Visualize Your Model with Keras and TensorBoard

In [2]:
from keras.models import Model
from keras.layers import Convolution2D, BatchNormalization, MaxPooling2D, Flatten, Dense
from keras.layers import Input, Dropout
from keras.layers.advanced_activations import ELU
from keras.regularizers import l2

import tensorflow as tf

from settings import *
from train import _get_extended_data, _load_features, _batch_generator, _solid_full_data

In [3]:
def build_model(n_classes):

    with tf.name_scope('input'):
        input_shape = (N_MEL_BANDS, SEGMENT_DUR, 1)
        channel_axis = 3
        melgram_input = Input(shape=input_shape)

        m_size = 70
        n_size = 3
        n_filters = 64
        maxpool_const = 4

    with tf.name_scope('layer1'):
        x = Convolution2D(n_filters, m_size, n_size,
                          border_mode='same',
                          init='he_normal',
                          W_regularizer=l2(1e-5))(melgram_input)

        x = BatchNormalization(axis=channel_axis, mode=0)(x)
        x = ELU()(x)
        x = MaxPooling2D(pool_size=(N_MEL_BANDS, SEGMENT_DUR/maxpool_const))(x)
        x = Flatten()(x)

    with tf.name_scope('layer2'):
        x = Dropout(0.5)(x)
        x = Dense(n_classes, init='he_normal', W_regularizer=l2(1e-5), activation='softmax', name='prediction')(x)

    with tf.name_scope('final_model'):
        model = Model(melgram_input, x)

    return model

In [11]:
dataset = pd.read_csv(IRMAS_TRAINING_META_PATH, names=["filename", "class_id"])
X_train, X_val, y_train, y_val = train_test_split(list(dataset.filename),
                                                  to_categorical(np.array(dataset.class_id, dtype=int)),
                                                  test_size=VALIDATION_SPLIT)
extended_x_train, extended_y_train = _get_extended_data(X_train, y_train)
extended_x_val, extended_y_val = _get_extended_data(X_val, y_val)
y_train = extended_y_train
y_val = extended_y_val
X_train = _load_features(extended_x_train)
X_val = _load_features(extended_x_val)

In [12]:
def train(X_train, X_val, y_train, y_val):
    optimizer = SGD(lr=init_lr, momentum=0.9, nesterov=True)

    model = build_model(IRMAS_N_CLASSES)

    early_stopping = EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_EPOCH)
    save_clb = ModelCheckpoint(
        "{weights_basepath}/".format(
            weights_basepath=MODEL_WEIGHT_BASEPATH) +
        "epoch.{epoch:02d}-val_loss.{val_loss:.3f}",
        monitor='val_loss',
        save_best_only=True)
    tb = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_grads=True)
    tb.validation_data = _solid_full_data(X_val, y_val)

    model.summary()
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit_generator(_batch_generator(X_train, y_train),
                        samples_per_epoch=SAMPLES_PER_EPOCH,
                        nb_epoch=MAX_EPOCH_NUM,
                        verbose=2,
                        callbacks=[save_clb, early_stopping, tb],
                        validation_data=_solid_full_data(X_val, y_val),
                        nb_val_samples=SAMPLES_PER_VALIDATION,
                        class_weight=None,
                        nb_worker=1)

In [ ]:
train(X_train, X_val, y_train, y_val)